In [1]:
from tools import *

from tqdm import tqdm
import json
import requests
from tqdm import tqdm

import json
import requests

import tools 
from typing import List, Dict, Tuple


In [2]:
stats = tools.clear_all_graph_data()


🗑️  Starting to clear all graph data...
   Deleting all relationships...
   Deleting all nodes...
   Cleaning up GDS graphs...
   - No 'entity' graph to drop
   Dropping all indexes...
   - Dropped index: entities
   Dropping all constraints...
✅ Graph cleanup completed!
   - Deleted 1338 relationships
   - Deleted 123 nodes
   - Remaining nodes: 0


In [3]:
url = "https://www.gutenberg.org/cache/epub/1727/pg1727.txt"
response = requests.get(url)

In [4]:
def chunk_into_books(text: str) -> List[str]:
    return (
        text.split("PREFACE TO FIRST EDITION")[2]
        .split("FOOTNOTES")[0]
        .strip()
        .split("\nBOOK")[1:]
    )

books = chunk_into_books(response.text)

In [5]:
token_count = [num_tokens_from_string(el) for el in books]
print(
    f"""There are {len(token_count)} books with token sizes:
- avg {sum(token_count) / len(token_count)}
- min {min(token_count)}
- max {max(token_count)}
"""
)

There are 24 books with token sizes:
- avg 6515.208333333333
- min 4459
- max 10760



In [6]:
chunked_books = [chunk_text(book, 1000, 40) for book in books]

In [7]:
len(chunked_books[0])

22

In [8]:
from typing import List, Dict, Tuple

ENTITY_TYPES = "PERSON,ORGANIZATION,LOCATION,GOD,EVENT,CREATURE,WEAPON_OR_TOOL"

def extract_entities(text: str) -> Tuple[List[Dict], List[Dict]]:
    """
    Extract entities and relationships from text using Bedrock
    
    Args:
        text: Input text to analyze
        
    Returns:
        Tuple of (entities, relationships) as lists of dictionaries
    """
    # Construct prompt
    messages = [
        {"role": "user", "content": tools.create_extraction_prompt(ENTITY_TYPES, text)}
    ]
    
    # Make the LLM call using Bedrock (model parameter is ignored)
    output = tools.chat(messages, model="gpt-4o")  # Will use Bedrock instead
    
    # Parse output and return both entities and relationships
    return tools.parse_extraction_output(output)

In [9]:
number_of_books = 1
for book_i, book in enumerate(
    tqdm(chunked_books[:number_of_books], desc="Processing Books")
):
    for chunk_i, chunk in enumerate(tqdm(book, desc=f"Book {book_i}", leave=False)):
        nodes, relationships = extract_entities(chunk)
        neo4j_driver.execute_query(
            tools.import_nodes_query,
            data=nodes,
            book_id=book_i,
            text=chunk,
            chunk_id=chunk_i,
        )
        neo4j_driver.execute_query(
            tools.import_relationships_query, data=relationships
        )

Book 0:   0%|          | 0/22 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";MUSE;GOD;Divine being invoked to tell the story of the ingenious hero who travelled far and wide after sacking Troy)
|
("entity";TROY;LOCATION;Famous town that was sacked by the ingenious hero)
|
("entity";HYPERION;GOD;Sun-god whose cattle were eaten by the hero's men, leading to their destruction)
|
("entity";JOVE;GOD;Father deity, referred to as the father of the Muse)
|
("entity";ULYSSES;PERSON;Ingenious hero who travelled far and wide after sacking Troy, trying to return home to his wife and country)
|
("entity";CALYPSO;GOD;Goddess who detained Ulysses in a large cave and wanted to marry him)
|
("relationship";MUSE;JOVE;The Muse is described as the daughter of Jove;8)
|
("relationship";ULYSSES;TROY;Ulysses sacked the famous town of Troy;9)
|
("relationship";ULYSSES;HYPERION;Ulysses's men perished after eating Hyperion's cattle, which prevented their return home;7)
|
("relationship";HYPERION;ULYSSES;H


Book 0:   5%|▍         | 1/22 [00:07<02:42,  7.72s/it]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";ITHACA;LOCATION;The homeland where the hero is destined to return after years of wandering)
|
("entity";NEPTUNE;GOD;God who persecuted the hero without ceasing and would not let him get home, later went to the Ethiopians for a festival)
|
("entity";ETHIOPIANS;PERSON;People at the world's end who lie in two halves, one looking West and the other East, who received Neptune for a festival)
|
("entity";OLYMPIAN JOVE;GOD;Sire of gods and men who spoke first when the gods met in his house, also called the father of gods)
|
("entity";AEGISTHUS;PERSON;Man who made love to Agamemnon's wife unrighteously and killed Agamemnon, later killed by Orestes)
|
("entity";AGAMEMNON;PERSON;Man who was killed by Aegisthus, father of Orestes)
|
("entity";ORESTES;PERSON;Son of Agamemnon who killed Aegisthus in revenge for his father's murder)
|
("entity";AGAMEMNON'S WIFE;PERSON;Woman who was seduced by Aegisthus, leading to the


Book 0:   9%|▉         | 2/22 [00:17<03:02,  9.15s/it]

...


Book 0:  91%|█████████ | 20/22 [03:44<00:20, 10.50s/it]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";TELEMACHUS;PERSON;Young man who declares his father dead and identifies the stranger as Mentes, though he knows in his heart it was a goddess)
|
("entity";ULYSSES;PERSON;Telemachus' dead father who was an old friend of Mentes)
|
("entity";PENELOPE;PERSON;Telemachus' mother who sometimes sends for soothsayers and questions them about prophecies)
|
("entity";SOOTHSAYER;PERSON;Prophet whom Penelope consults but whose prophecies Telemachus does not heed)
|
("entity";MENTES;PERSON;Son of Anchialus and chief of the Taphians, identified as the stranger and old friend of Ulysses)
|
("entity";ANCHIALUS;PERSON;Father of Mentes)
|
("entity";TAPHIANS;PERSON;People ruled by Mentes as their chief)
|
("entity";MINERVA;GOD;Goddess who Telemachus knew in his heart had been the stranger disguised as Mentes)
|
("entity";SUITORS;PERSON;Men who returned to singing and dancing until evening, then went home to bed)
|
("entity"


Book 0:  95%|█████████▌| 21/22 [04:00<00:12, 12.09s/it]

 Telemachus;7)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";LAERTES;PERSON;Man who respected Euryclea as much as his own wedded wife but did not take her to his bed for fear of his wife's resentment)
|
("entity";EURYCLEA;PERSON;Good old woman who lighted Telemachus to his room, loved him better than other women in the house, and had nursed him as a baby)
|
("entity";LAERTES' WIFE;PERSON;Wedded wife of Laertes whose resentment he feared regarding Euryclea)
|
("entity";TELEMACHUS;PERSON;Young man who was nursed by Euryclea as a baby and now lies thinking of his intended voyage and Minerva's counsel)
|
("entity";MINERVA;GOD;Goddess who gave counsel to Telemachus about his intended voyage)
|
("entity";SHIRT;WEAPON_OR_TOOL;Garment that Telemachus took off and gave to Euryclea to fold and hang)
|
("entity";SILVER CATCH;WEAPON_OR_TOOL;Silver mechanism used to pull the door closed)
|
("entity";WOOLLEN FLEECE;WEAPON_OR_TOOL;Covering under which Telemachus la


Processing Books: 100%|██████████| 1/1 [04:11<00:00, 251.40s/it]


In [10]:
nodes[0]

{'record_type': 'entity',
 'entity_name': 'LAERTES',
 'entity_type': 'PERSON',
 'entity_description': "Man who respected Euryclea as much as his own wedded wife but did not take her to his bed for fear of his wife's resentment"}

In [11]:
relationships[0]

{'record_type': 'relationship',
 'source_entity': 'LAERTES',
 'target_entity': 'EURYCLEA',
 'relationship_description': "Laertes respected Euryclea as much as his own wife but did not take her to his bed for fear of his wife's resentment",
 'relationship_strength': 8}

In [12]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (:`__Entity__`)
    RETURN 'entity' AS type, count(*) AS count
    UNION
    MATCH ()-[:RELATIONSHIP]->()
    RETURN 'relationship' AS type, count(*) AS count
    """
)
print([el.data() for el in data])

[{'type': 'entity', 'count': 96}, {'type': 'relationship', 'count': 203}]


In [13]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
WHERE n.name = "ORESTES"
RETURN n.description AS description"""
)
print([el.data()['description'] for el in data])

[["Son of Agamemnon who killed Aegisthus in revenge for his father's murder", 'Son who would grow up to take revenge on Aegisthus for killing his father Agamemnon', "Hero praised for killing his father's murderer Aegisthus, held up as an example for Telemachus"]]


In [14]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:__Entity__)-[:RELATIONSHIP]-(m:__Entity__)
WITH n,m, count(*) AS countOfRels
ORDER BY countOfRels DESC LIMIT 1
MATCH (n)-[r:RELATIONSHIP]-(m)
RETURN n.name AS source, m.name AS target, countOfRels, collect(r.description) AS descriptions
"""
)
print([el.data() for el in data])

[{'source': 'TELEMACHUS', 'target': 'ULYSSES', 'countOfRels': 14, 'descriptions': ['Telemachus is the son of Ulysses and declares his father dead and gone', 'Telemachus is the son of Ulysses and heir to those whom his father won', 'Telemachus is the son of Ulysses and heir to his position as chief', "Telemachus thinks more than ever about his father after Minerva's encouragement", 'Telemachus is searching for news of his missing father Ulysses', 'Telemachus should go in quest of his father Ulysses who has long been missing', 'Telemachus is the son of Ulysses and thinks about his brave father', 'Telemachus is the son of Ulysses, referred to as his unhappy father', 'Telemachus refers to his absent father Ulysses whose bones lie rotting and at whose cost the banquet is held', "Telemachus is the son of Ulysses and speaks about his father's absence and ill fate", 'Telemachus resembles Ulysses about the head and eyes', 'Telemachus is the son of Ulysses and would have been heir to his renown'

In [ ]:
candidates_to_summarize, _, _ = neo4j_driver.execute_query(
    """MATCH (e:__Entity__) WHERE size(e.description) > 1 
    RETURN e.name AS entity_name, e.description AS description_list"""
)
summaries = []
for candidate in tqdm(candidates_to_summarize, desc="Summarizing entities"):
    messages = [
        {
            "role": "user",
            "content": tools.get_summarize_prompt(
                candidate["entity_name"], candidate["description_list"]
            ),
        },
    ]
    summary = chat(messages, model="gpt-4o")
    summaries.append({"entity": candidate["entity_name"], "summary": summary})

tools.import_entity_summary(summaries, neo4j_driver)

Summarizing entities:   0%|          | 0/28 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
Telemachus is the son of Ulysses, bearing a strong physical resemblance to his father about the head and eyes. As a young man grieving for his lost father, Telemachus sits moodily among the suitors who have invaded his house under the pretext of courting his mother Penelope. These chiefs from various islands are consuming his family's resources, causing him great suffering as he thinks about his brave father and hopes for his return, though he publicly declares his father dead.

When the goddess Minerva visits him disguised as Mentes, chief of the Taphians, Telemachus serves as a gracious host, welcoming the guest, taking her spear, and providing seating away from the suitors. He speaks privately with Minerva about the cost of the banquet and questions the stranger about their identity and origin. Though he identifies the visitor as Mentes to others, Telemachus knows in his heart that the stranger was actually a g

Summarizing entities:   4%|▎         | 1/28 [00:11<05:01, 11.16s/it]

 legacy.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The Achaeans are Greek warriors who fought in the war at Troy and subsequently faced a difficult return journey home. During their return from Troy, the Achaeans suffered various ills inflicted upon them by the goddess Minerva. Among these warriors, Menelaus was notably the last of the Achaeans to successfully make it home from the war.

The Achaeans maintain a connection to Telemachus and his household, as Telemachus plans to call them in assembly to address his situation with the suitors. They represent a community that honors fallen heroes, as they would have built a commemorative mound over Ulysses' ashes if he had died honorably in battle rather than disappearing without a trace. This demonstrates their respect for proper burial rites and the commemoration of their fellow warriors who served alongside them in

Summarizing entities:   7%|▋         | 2/28 [00:17<03:42,  8.57s/it]

...

 to his marriage and considerate of his wife's feelings.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The ship is a vessel that plays a central role in the narrative's travel plans and divine deception. When the goddess Minerva disguises herself as Mentes, she claims ownership of a ship that lies in the harbour Rheithron, using this as part of her credible identity as a traveling merchant. However, since Minerva is actually the one speaking, this ship effectively belongs to her in the context of her assumed identity, and she references needing to return to this vessel as part of maintaining her disguise.

Additionally, there is guidance given that Telemachus should obtain the best available ship for his own quest, specifically one that can accommodate a crew of twenty men, to search for news of his missing father Ulysses. This represents the practical vessel that Telemachus will need for his journey to Pylos and Sparta, separate from the ship associated with 

Summarizing entities:  50%|█████     | 14/28 [02:32<02:17,  9.80s/it]

 heroic journey to find his father.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The crew represents two distinct groups of twenty men in the narrative. First, there is the crew that Telemachus should assemble to accompany him on his quest to find his father - twenty men who would sail with him to Pylos and Sparta as he searches for news of Ulysses. Second, there is the crew that belongs to Minerva in her disguise as Mentes, who she claims will become impatient if kept waiting too long at the harbor. This second crew serves as part of Minerva's cover story, providing a credible reason for her to limit the length of her visit and maintain the authenticity of her assumed identity as a traveling merchant who must attend to her ship and sailors. Both crews represent the practical necessity of having sufficient manpower to operate ships for long voyages, whether for divine deception or heroic qu

Summarizing entities:  54%|█████▎    | 15/28 [02:39<01:55,  8.90s/it]

ests.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)


In [ ]:
result, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
WHERE n.summary IS NULL
RETURN n.name AS name""")

In [ ]:
result

In [ ]:
summary, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
WHERE n.name = "ORESTES"
RETURN n.summary AS summary""")
print(summary[0]['summary'])

In [ ]:
# 방법 2: 더 안전한 버전

def get_person_summary(person_name: str):
    """Get summary for a specific person from Neo4j"""
    try:
        query = """
        MATCH (n:PERSON)
        WHERE n.name = $person_name
        RETURN n.summary AS summary
        """
        
        records, _, _ = tools.neo4j_driver.execute_query(
            query, 
            person_name=person_name
        )
        
        if records and len(records) > 0:
            summary = records[0]['summary']
            if summary:
                return summary
            else:
                return f"No summary available for {person_name}"
        else:
            return f"Person '{person_name}' not found in database"
            
    except Exception as e:
        return f"Error querying database: {e}"

# 사용 예시
result = get_person_summary("ORESTES")
print(result)


In [ ]:
rels_to_summarize, _, _ = neo4j_driver.execute_query(
    """MATCH (s:__Entity__)-[r:RELATIONSHIP]-(t:__Entity__)
    WHERE id(s) < id(t)
    WITH s.name AS source, t.name AS target, 
           collect(r.description) AS description_list,
           count(*) AS count
    WHERE count > 1
    RETURN source, target, description_list"""
)
rel_summaries = []
for candidate in tqdm(rels_to_summarize, desc="Summarizing relationships"):
    entity_name = f"{candidate['source']} relationship to {candidate['target']}"
    messages = [
        {
            "role": "user",
            "content": tools.get_summarize_prompt(
                entity_name, candidate["description_list"]
            ),
        },
    ]
    summary = chat(messages, model="gpt-4o")
    rel_summaries.append({"source": candidate["source"], "target": candidate["target"], "summary": summary})

tools.import_rels_summary(summaries, neo4j_driver)

In [ ]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:__Entity__)-[r:SUMMARIZED_RELATIONSHIP]-(m:__Entity__)
WHERE n.name = 'TELEMACHUS' AND m.name = 'MINERVA'
RETURN r.summary AS description
"""
)
print(data[0]["description"])

In [ ]:
community_distribution = tools.calculate_communities(neo4j_driver)
print(f"There are {community_distribution['communityCount']} communities with distribution: {community_distribution['communityDistribution']}")

In [ ]:
community_info, _, _ = neo4j_driver.execute_query(tools.community_info_query)

In [ ]:
import boto3
import json
from langchain_aws import ChatBedrock
from langchain.schema import HumanMessage
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class Finding(BaseModel):
    summary: str = Field(description="Finding summary")
    explanation: str = Field(description="Detailed explanation")

class LiteraryAnalysis(BaseModel):
    title: str = Field(description="Analysis title")
    summary: str = Field(description="Analysis summary")
    rating: float = Field(description="Rating score")
    rating_explanation: str = Field(description="Rating explanation")
    findings: List[Finding] = Field(description="List of findings")

class AdvancedLLM:
    """고급 LLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
        self.parser = PydanticOutputParser(pydantic_object=LiteraryAnalysis)
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 4000,
                    "temperature": 0.15,
                    "top_p": 0.9,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None
    
    def generate_minerva_telemachus_analysis(self, summary):
        """커뮤니티 요약을 기반으로 분석 생성"""
        if not self.llm:
            return None
        
        prompt = f"""
        다음 커뮤니티 요약을 분석하여 JSON 형식으로 작성해주세요:
    
        커뮤니티 요약: {summary}
    
        {self.parser.get_format_instructions()}
        """
        
        try:
            message = HumanMessage(content=prompt)
            response = self.llm([message])
            parsed_response = self.parser.parse(response.content)
            return parsed_response.dict()
        except Exception as e:
            print(f"❌ 분석 생성 실패: {e}")
            return None
                 

In [ ]:
llm=AdvancedLLM()

In [ ]:
communities = []
for community in tqdm(community_info, desc="Summarizing communities"):
    messages = [
        {
            "role": "user",
            "content": tools.get_summarize_community_prompt(
                community["nodes"], community["rels"]
            ),
        },
    ]
    summary = chat(messages, model="gpt-4o")
    re_summary=llm.generate_minerva_telemachus_analysis(summary)
    communities.append(
        {
            "community": re_summary,
            "communityId": community["communityId"],
            "nodes": [el["id"] for el in community["nodes"]],
        }
    )

neo4j_driver.execute_query(tools.import_community_query, data=communities)

In [ ]:
neo4j_driver.execute_query("""UNWIND $data AS row
                            MERGE (c:__Community__ {communityId: row.communityId})
                            SET c.title = row.community.title,
                                c.summary = row.community.summary,
                                c.rating = row.community.rating,
                                c.rating_explanation = row.community.rating_explanation
                            WITH c, row
                            UNWIND row.nodes AS node
                            MERGE (n:__Entity__ {name: node})
                            MERGE (n)-[:IN_COMMUNITY]->(c)"""
                           , data=communities)

In [ ]:
result = neo4j_driver.execute_query(tools.import_community_query, data=communities)
print(f"Nodes created: {result.summary.counters.nodes_created}")
print(f"Relationships created: {result.summary.counters.relationships_created}")
print(f"Properties set: {result.summary.counters.properties_set}")

In [ ]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (c:__Community__)
WITH c, count {(c)<-[:IN_COMMUNITY]-()} AS size
ORDER BY size DESC LIMIT 1
RETURN c.title AS title, c.summary AS summary
"""
)
print(data[0]["title"])
print(data[0]["summary"])

In [ ]:
def global_retriever(query: str, rating_threshold: float = 5) -> str:
    community_data, _, _ = neo4j_driver.execute_query(
        """
    MATCH (c:__Community__)
    WHERE c.rating >= $rating
    RETURN c.summary AS summary
    """,
        rating=rating_threshold,
    )
    print(f"Got {len(community_data)} community summaries")
    intermediate_results = []
    for community in tqdm(community_data, desc="Processing communities"):
        intermediate_messages = [
            {
                "role": "system",
                "content": tools.get_map_system_prompt(community["summary"]),
            },
            {
                "role": "user",
                "content": query,
            },
        ]
        intermediate_response = chat(intermediate_messages, model="gpt-4o")
        intermediate_results.append(intermediate_response)

    final_messages = [
        {
            "role": "system",
            "content": tools.get_reduce_system_prompt(intermediate_results),
        },
        {"role": "user", "content": query},
    ]
    summary = chat(final_messages, model="gpt-4o")
    return summary

In [ ]:
print(global_retriever("What is this story about?"))

In [ ]:
entities, _, _ = neo4j_driver.execute_query(
    """
MATCH (e:__Entity__)
RETURN e.summary AS summary, e.name AS name
"""
)



In [ ]:
entities[0]

In [ ]:
data = [{"name": el["name"], "embedding": embed(el["summary"])} for el in entities]


In [ ]:
data[0]['name']

In [ ]:
neo4j_driver.execute_query(
    """
UNWIND $data AS row
MATCH (e:__Entity__ {name: row.name})
CALL db.create.setNodeVectorProperty(e, 'embedding', row.embedding)
""",
    data=data,
)

In [ ]:
neo4j_driver.execute_query(
    """
CREATE VECTOR INDEX entities IF NOT EXISTS
FOR (n:__Entity__)
ON (n.embedding)
""",
    data=data,
)

In [ ]:
# Check embedding values (first few dimensions)
result = neo4j_driver.execute_query(
    "MATCH (e:__Entity__) WHERE e.embedding IS NOT NULL RETURN e.name, e.embedding[0..3] as first_dims LIMIT 3"
)

In [ ]:
result

In [ ]:
local_search_query = """
CALL db.index.vector.queryNodes('entities', $k, $embedding)
YIELD node, score
WITH collect(node) as nodes
WITH collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.text AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
collect {
    UNWIND nodes as n
    MATCH (n)-[r:SUMMARIZED_RELATIONSHIP]-(m) 
    WHERE m IN nodes
    RETURN r.summary AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
collect {
    UNWIND nodes as n
    RETURN n.summary AS descriptionText
} as entities
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: insideRels, 
       Entities: entities} AS text
"""

In [ ]:
k_entities = 5

topChunks = 3
topCommunities = 3
topInsideRels = 3


def local_search(query: str) -> str:
    context, _, _ = neo4j_driver.execute_query(
        local_search_query,
        embedding=embed(query),
        topChunks=topChunks,
        topCommunities=topCommunities,
        topInsideRels=topInsideRels,
        k=k_entities,
    )
    context_str = str(context[0]["text"])
    local_messages = [
        {
            "role": "system",
            "content": tools.get_local_system_prompt(context_str),
        },
        {
            "role": "user",
            "content": query,
        },
    ]
    final_answer = chat(local_messages, model="gpt-4o")
    return final_answer


In [ ]:
print(local_search("Who is Ulysses?"))